In [ ]:
# standard imports
import numpy as np
import xarray as xr
import rioxarray
import matplotlib.pyplot as plt

# load up the saved dataset
import pickle

# add module to path so python can import it. This is the directory with 
# the __init__.py file in it to let python know this is a module.
import sys
sys.path.append('/Users/zachkeskinen/Documents/spicy-snow')

# import the function to test
from spicy_snow.download.sentinel1 import s1_img_search, hyp3_pipeline, download_hyp3, combine_s1_images
import shapely
import os
# sets dates and areas
dates = ('2019-02-01', '2019-02-25')
area = shapely.geometry.box(-115, 43, -114, 44)

# Set skip = True to skip indented steps
skip = False


os.makedirs('../data' , exist_ok = True)
# get asf_search search results
search_results = s1_img_search(area, dates)
print(f'Found {len(search_results)} results')

In [ ]:
# download s1 images into dataset ['s1'] keyword
jobs = hyp3_pipeline(search_results, job_name = 'improve-downloads', existing_job_name = 'improve-downloads')
das = download_hyp3(jobs, area, outdir='/Users/zachkeskinen/Documents/spicy-snow/data/tmp4', clean = False)
ds = combine_s1_images(das)

In [ ]:
from spicy_snow.download.snow_cover import download_snow_cover
from spicy_snow.download.forest_cover import download_fcf

In [ ]:
ds = download_snow_cover(ds)

In [ ]:
ds = download_fcf(ds, '/Users/zachkeskinen/Documents/spicy-snow/data/fcf.tif')

In [ ]:
with open('/Users/zachkeskinen/Documents/spicy-snow/tests/test_data/2_img_ds', 'wb') as f:
    pickle.dump(ds, f)